In [1]:
# === 라이브러리 임포트 ===
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# === 1. 문서 로딩: PDF 읽기 ===
print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('./data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

# === 2. 문서 분할 ===
print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,      # 로컬 모델 대응 크기
    chunk_overlap=150,   # 중복 최소화
    separators=["\n\n", "\n", ".", " ", ""]
)
chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")

# === 3. 임베딩 변환 (HuggingFace 사용) ===
print("==> 3. 벡터화 → 임베딩으로 변환")
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': 'cpu'},  # GPU 있으면 'cuda'
    encode_kwargs={'normalize_embeddings': True}
)

# === 4. 벡터 저장: FAISS ===
print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)

# === 5. 검색기 구성 ===
print("==> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # 유사 문서 4개만 가져옴
)

# === 6. 로컬 LLM 설정 (Qwen3) ===
print("==> 6. 생성 → LLM으로 답변 생성")
llm = ChatOllama(
    model="qwen3:1.7b",
    base_url="http://localhost:11434",  # Ollama 서버 주소
    temperature=0.1,
    num_predict=1500
)

# === 7. 프롬프트 정의 ===
prompt_template = PromptTemplate.from_template("""
너는 PDF 문서를 기반으로 질문에 정확하게 답하는 전문가야.

질문: {question}
=========
문서 내용:
{context}
=========
답변:""")

# === 8. QA 체인 구성 ===
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template},
    return_source_documents=True
)

# === 9. 질문 실행 ===
query = "이 문서의 주요 주제는 무엇인가요?"
print("==> 7. 질의 실행:")
result = qa_chain.invoke({"query": query})

print("\n🔍 [답변]")
print(result['result'])

print("\n📄 [참고 문서 수]")
print(len(result['source_documents']))


==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  76개 청크 생성 완료
==> 3. 벡터화 → 임베딩으로 변환


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==> 4. 저장 → FAISS 벡터스토어에 저장
==> 5. 검색 → 질문과 유사한 문서 찾기
==> 6. 생성 → LLM으로 답변 생성
==> 7. 질의 실행:

🔍 [답변]
<think>
Okay, let's see. The user is asking about the main theme of the provided document. First, I need to look through the content carefully.

The document starts with sections labeled 10.3 to 11.1, which seem to be chapters or sections. The first part (10.3) talks about the main method and other static methods, then 10.4 about document creation, 10.5 about working with libraries, 10.6 about creating objects from library classes. Then there's a section on summary (11.1) which is a quick reference. 

The reference section mentions that the tutorial's content is summarized in the 11th chapter, so the main points are covered there. The note about the main method and static methods in the first part, along with the sections on document creation and library usage, suggest that the document is about Java programming, specifically covering methods, libraries, and documentation processes.

The